<a href="https://colab.research.google.com/github/shu-nya/Image-Classification-of-White-Blood-Cells/blob/main/Classification_of_WBC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#pip install split-folders

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
from keras.models import Sequential
from keras import applications
from keras import optimizers
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Input, Conv2D, MaxPooling2D, Lambda
from keras.layers import Dense
from keras.models import Model
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import to_categorical
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn.utils import shuffle
import csv
import cv2
import scipy
import os
from PIL import Image
#import splitfolders
import shutil

In [3]:
num_classes = 9
epochs = 20

BASE_DIR = '/content/drive/MyDrive/taskMedPrime'

batch_size = 32

In [4]:
classes = ['bands', 'basophils', 'blasts', 'eosinophils', 'lymphocytes', 'metamyelocytes', 'monocytes', 'myelocytes', 'neutrophils']

for i in classes:

    os.makedirs(BASE_DIR + '/train/' + i)

    os.makedirs(BASE_DIR + '/test/' + i)

    source = BASE_DIR + '/wbc/' + i

    allFileNames = os.listdir(source)

    np.random.shuffle(allFileNames)

    test_ratio = 0.20

    train_FileNames, test_FileNames = np.split(np.array(allFileNames),
                                                      [int(len(allFileNames)* (1 - test_ratio))])


    train_FileNames = [source + '/' + name for name in train_FileNames.tolist()]
    test_FileNames = [source + '/' + name for name in test_FileNames.tolist()]


    for name in train_FileNames:
        shutil.copy(name, BASE_DIR +'/train/' + i)

    for name in test_FileNames:
        shutil.copy(name, BASE_DIR +'/test/' + i)

In [6]:
#splitfolders.fixed(BASE_DIR + '/' + 'wbc', output="output", seed=1337, fixed=(100), oversample=True, group_prefix=None, move=True)

In [7]:
#splitfolders.ratio(BASE_DIR + '/' + 'wbc', output="output",
#    seed=1337, ratio=(.8, .2), group_prefix=None, move=True)

In [5]:
def get_data(folder):
    X = []
    y = []

    for wbc_type in os.listdir(folder):
        if not wbc_type.startswith('.'):

            for image_filename in os.listdir(folder + '/' + wbc_type):
                img_file = cv2.imread(folder + '/' + wbc_type + '/' + image_filename)

                if img_file is not None:
                    img_arr = np.asarray(img_file)
                    X.append(img_arr)
                    y.append(wbc_type)

    X = np.asarray(X)
    y = np.asarray(y)
    return X,y

In [6]:
X_train, y_train = get_data(BASE_DIR + '/train')
X_test, y_test = get_data(BASE_DIR + '/test')


X_train = X_train * 1./255.
X_test = X_test * 1./255.


In [7]:
print(len(X_train))

421


In [8]:
print(len(y_train))

421


In [9]:
print(len(X_test))

107


In [10]:
print(len(y_test))

107


In [9]:
#Convert labels to one hot encoding

encoder = LabelEncoder()
encoder.fit(y_train)
encoded_y_train = encoder.transform(y_train)
encoded_y_test = encoder.transform(y_test)

y_train = np_utils.to_categorical(encoded_y_train)
y_test = np_utils.to_categorical(encoded_y_test)


In [15]:
print(y_train)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


In [16]:
print(y_test)

[[1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0.

In [17]:
def get_model():
    model = Sequential()
    model.add(Lambda(lambda x: x/127.5 - 1., input_shape=(256, 256, 3), output_shape=(256, 256, 3)))
    model.add(Conv2D(32, (3, 3), input_shape=(256, 356, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.7))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                optimizer='Adam',
                metrics=['accuracy'])

    return model

In [22]:
def get_model():
    model = Sequential()
    model.add(Lambda(lambda x: x/127.5 - 1., input_shape=(256, 256, 3), output_shape=(256, 256, 3)))

    # layer 1
    model.add(Conv2D(32, (3, 3), input_shape=(256, 356, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(rate=0.25))

    # layer 2
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(rate=0.25))


#    model.add(Conv2D(64, (3, 3)))
#    model.add(Activation('relu'))
#    model.add(MaxPooling2D(pool_size=(2, 2)))


    # Fully connected network layer
    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.25))

    # Output
    model.add(Dense(9))
    model.add(Activation('softmax'))

    # Compile
    model.compile(loss='categorical_crossentropy',
                optimizer='Adam',
                metrics=['accuracy'])

    return model

In [10]:
def get_model():
    model = Sequential()
    model.add(Lambda(lambda x: x/127.5 - 1., input_shape=(256, 256, 3), output_shape=(256, 256, 3)))

    # layer 1
    model.add(Conv2D(32, kernel_size=(3, 3), padding='same', input_shape=(256, 356, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(rate=0.25))

    # layer 2
    model.add(Conv2D(32, kernel_size=(3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(rate=0.40))

    # layer 3
    model.add(Conv2D(64, kernel_size=(3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(rate=0.40))


    # Fully connected network layer
    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.40))

    # Output
    model.add(Dense(9))
    model.add(Activation('softmax'))

    # Compile
    model.compile(loss='categorical_crossentropy',
                optimizer='Adam',
                metrics=['accuracy'])

    return model

In [12]:
#Input
x_input = Input(shape=X_test.shape[1:])

# Layer 1
x = Conv2D(filters=32, kernel_size=3, activation='relu')(x_input)
x = MaxPooling2D(pool_size=2)(x)
x = Dropout(rate=0.25)(x)

# Layer 2
x = Conv2D(filters=64, kernel_size=3, activation='relu')(x)
x = MaxPooling2D(pool_size=2)(x)
x = Dropout(rate=0.25)(x)

# Fully Connected Network layer
x = Flatten()(x)
x = Dense(units=128, activation='relu')(x)
x = Dropout(rate=0.25)(x)

# Output
x = Dense(units=4, activation='softmax')(x)

# Build model
model1 = Model(inputs=x_input, outputs=x)

In [13]:
# Compile and Train Model

model1.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
history1 = model1.fit(x=X_train, y=y_train, epochs=epoch, batch_size=64, validation_split=0.1, verbose=2)

Epoch 1/150


ValueError: ignored

In [ ]:
# Test Trained Model on Test Data

y1_predict = model1.predict(X_test).argmax(axis=1)
model1_test_acc = (y1_predict == y_test).sum()/(y1_predict.shape[0])

In [11]:
model = get_model()

# fits the model on batches
model.fit(
    X_train,
    y_train,
    validation_split=0.1,
    epochs=150,
    shuffle=True,
    batch_size=64,
    verbose=2)

model.save_weights('balanced_model_2.h5')

Epoch 1/150
6/6 - 16s - loss: 5.6501 - accuracy: 0.2513 - val_loss: 2.1382 - val_accuracy: 0.0000e+00 - 16s/epoch - 3s/step
Epoch 2/150
6/6 - 1s - loss: 1.8418 - accuracy: 0.2857 - val_loss: 1.9497 - val_accuracy: 0.0000e+00 - 1s/epoch - 195ms/step
Epoch 3/150
6/6 - 1s - loss: 1.7592 - accuracy: 0.3333 - val_loss: 1.8972 - val_accuracy: 0.0000e+00 - 1s/epoch - 179ms/step
Epoch 4/150
6/6 - 1s - loss: 1.7381 - accuracy: 0.3783 - val_loss: 1.9598 - val_accuracy: 0.0000e+00 - 1s/epoch - 174ms/step
Epoch 5/150
6/6 - 1s - loss: 1.7477 - accuracy: 0.3624 - val_loss: 1.9621 - val_accuracy: 0.0000e+00 - 1s/epoch - 175ms/step
Epoch 6/150
6/6 - 1s - loss: 1.7551 - accuracy: 0.3651 - val_loss: 2.0299 - val_accuracy: 0.0000e+00 - 1s/epoch - 177ms/step
Epoch 7/150
6/6 - 1s - loss: 1.7215 - accuracy: 0.3889 - val_loss: 1.8967 - val_accuracy: 0.0000e+00 - 1s/epoch - 173ms/step
Epoch 8/150
6/6 - 1s - loss: 1.7263 - accuracy: 0.3466 - val_loss: 2.0645 - val_accuracy: 0.0000e+00 - 1s/epoch - 173ms/step
E

In [12]:
from sklearn.metrics import accuracy_score

print('Predicting on test data')

y_pred = np.rint(model.predict(X_test))

print(accuracy_score(y_test, y_pred))

Predicting on test data
4/4 [==============================] - 1s 73ms/step
0.0


In [13]:
from sklearn.metrics import confusion_matrix

y_pred_unencoded = np.argmax(y_pred, axis=1)

y_test_unencoded = np.argmax(y_test, axis=1)

print(confusion_matrix(y_test_unencoded, y_pred_unencoded))

[[ 6  0  0  0  0  0  0  0  0]
 [ 2  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  0  0]
 [ 5  0  0  0  0  0  0  0  0]
 [38  0  0  0  0  0  0  0  0]
 [ 5  0  0  0  0  0  0  0  0]
 [ 6  0  0  0  0  0  0  0  0]
 [ 2  0  0  0  0  0  0  0  0]
 [42  0  0  0  0  0  0  0  0]]
